In [1]:
# General imports
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

In [2]:
from qiskit import *
from qiskit.visualization import *
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session
from qiskit.primitives import Sampler
# Loading IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum", token="8558f42abd5e4c37e51ce37375293303fab09a45f771700ee214028841a7a19c5a8eedca85b84770d7d51177d9869776d88e8adb34bbf0a91b6c607a2e5d1a7")
backend = service.least_busy(operational=True, simulator=False)

In [459]:
############################################################# VARIABLES #############################################################################
B = 100 # Budget
R = 0.5 # Risk Ratio
delta = 30 # Penalty Term

In [ ]:
######################################################### IMPORT DATA HERE ##########################################################################
# panda #
import pandas as pds

df = pd.read_csv('sample.csv')

holder = []
for i in range(1,2):
    holder.append(df.iloc[:, i].tolist())

map_everything_for_me_please_here_are_all_the_prices(holder)
rev_the_hamiltonian()

In [ ]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.circuit import Parameter, QuantumCircuit

with Session(backend=backend) as session:
    E = []
    size = sum(d)
    
    x0 = 2 * np.pi * np.random.random(size*8)
    result = minimize(cost_funct, x0, method='COBYLA', options={'tol': 1e-4, 'disp': True})
    error = result.fun
    print(error)

    print("Optimal parameters:", result.x)
    print("Minimum function value:", result.fun)

In [529]:
def cost_funct(params):
    q = QuantumRegister(size)
    qc = QuantumCircuit(q)
    
    ######################### ANSATZ LIVES HERE #######################################
    ansatz = EfficientSU2(size)    
    qc.compose(ansatz, inplace=True)
    ##################################################################################
    
    observables = [
    [SparsePauliOp(hi, h_)], [SparsePauliOp(ji, J__)], [SparsePauliOp(bi, pi_)]
    ]
    
    estimator = StatevectorEstimator()
    pub = (qc, observables, params)
    job = estimator.run([pub])
    result = job.result()

    qc.draw('mpl')
    E.append(sum(result[0].data.evs))
    return sum(result[0].data.evs)

In [ ]:
######################################################### GENERATING HAMILTONIAN ####################################################################

#####################################################################################################################################################
def rev_the_hamiltonian():
    I = []
    for i in range(sum(d)):
        I.append("I")
    
    ############### hi Zi ################
    h = (np.array([u])*np.array([P])/B)
    h_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            h_.append(h[0][i]*(2**j))
        
    hi = []
    for i in range(sum(d)):
        I[i] = "Z"
        hi.append(stitch(I))
        I[i] = "I"
    ######################################
    
    ############# ji Zi Zj ###############
    J = ((convolu*(np.array([P])*(R/B)).reshape(-1, 1)).T)*(np.array([P])/B).reshape(-1, 1)
    J_ = np.dot(conver.T,(np.dot(J, conver)))
    
    J__ = []
    ji = []
    for i in range(sum(d)):
        for j in range(sum(d)):
            I[i] = "Z"
            I[j] = "Z"
            if j != i:
                ji.append(stitch(I))
                J__.append(J_[i,j])
            I[i] = "I"
            I[j] = "I"
    ######################################
    
    ########### Budget Penalty ###########
    pi = (np.array([P])*delta/B)
    pi_ = []
    for i in range(len(d)):
        for j in range(d[i]):
            pi_.append(pi[0][i]*(2**j))
    
    bi = copy.copy(hi)
        
    bi.append("I"*sum(d))
    pi_.append(-1) # -1 term
    ######################################

In [497]:
def map_everything_for_me_please_here_are_all_the_prices(holder):
    global p, P, r, u, convolu, d, conver
    
    # PRICE
    p = []
    for i in holder:
        p.append(i)
        
    # CURRENT PRICE
    P = []
    for i in holder:
        P.append(i[-1])
    
    # REVENUE
    r = []
    for i in holder:
        r.append(reven(i))

    # AVERAGE REVENUE
    u = []
    for i in r:
        u.append(avg(i))

    # CONVOLUTION MATRIX
    convolu = np.zeros((len(holder), len(holder)))

    for i in range(len(holder)):
        for j in range(len(holder)):
            convolu[i,j] = inner(add(r[i],-u[i]), add(r[j],-u[j]))

    # CONVERT TO BITS
    d = []
    for i in holder:
        d.append(round(np.log2(np.max(i))))

    # REVERSION MATRIX
    conver = np.zeros((len(d), sum(d)))
    
    k = 0
    for i in range(len(d)):
        for j in range(d[i]):
            conver[i,k] = 2**j
            k += 1

In [174]:
import copy

def norm(b):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] /= sum(v)
    return v, sum(v)

def avg(x):
    avg = sum(x)/len(x)
    return avg

def reven(v):
    b = []
    for i in range(1,len(v)):
        b.append(v[i]-v[i-1])
    return b

def add(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] += r
    return v

def multip(b, r):
    v = copy.copy(b)
    for i in range(len(v)):
        v[i] *= r
    return v

def inner(a, b):
    total = 0
    for i in range(len(a)):
        total += a[i] * b[i]
    return total

def stitch(v):
    s = ""
    for i in v:
        s += i
    return s